In [29]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


In [8]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


In [9]:
import os
from dotenv import load_dotenv

load_dotenv()


False

In [30]:
from langchain_ollama import ChatOllama
from langchain.chat_models import init_chat_model

llm = ChatOllama(model="llama3.2")
llm

ChatOllama(model='llama3.2')

In [31]:
## Node Functionality
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


In [32]:
graph_builder = StateGraph(State)

## Adding node
graph_builder.add_node("llmchatbot", chatbot)
## Adding Edges
graph_builder.add_edge(START, "llmchatbot")
graph_builder.add_edge("llmchatbot", END)

## compile the graph
graph = graph_builder.compile()


In [33]:
## Visualize the graph
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass


In [34]:
response = graph.invoke({"messages": "Hi"})
response.get("messages")[-1].content


'How can I assist you today?'

In [35]:
for event in graph.stream({"messages": "Hi How are you"}):
    for value in event.values():
        print(value['messages'][-1].content)


I'm just a language model, so I don't have feelings or emotions like humans do. However, I'm functioning properly and ready to help with any questions or tasks you may have! How can I assist you today?


In [46]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b

    Args:
        a (int): first int
        b (int): second int

    Returns:
        int: output int
    """
    return a * b

def addition(a: int, b: int) -> int:
    """Addition a and b

    Args:
        a (int): first int
        b (int): second int

    Returns:
        int: output int
    """
    return a + b


In [47]:
tools = [multiply, addition]
llm_with_tools=llm.bind_tools(tools)
llm_with_tools


RunnableBinding(bound=ChatOllama(model='llama3.2'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'Multiply a and b', 'parameters': {'properties': {'a': {'description': 'first int', 'type': 'integer'}, 'b': {'description': 'second int', 'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'addition', 'description': 'Addition a and b', 'parameters': {'properties': {'a': {'description': 'first int', 'type': 'integer'}, 'b': {'description': 'second int', 'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [39]:
## Stategraph
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition


## Node definition
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


## Grpah
builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

## Add Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", END)

## compile the graph
graph = builder.compile()


In [43]:
response = graph.invoke({"messages": "What is the multiplication of 2* 5 "})
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

What is the multiplication of 2* 5 
================================== Ai Message ==================================
Tool Calls:
  multiply (161f13d7-df6f-4931-9370-1523874fe28c)
 Call ID: 161f13d7-df6f-4931-9370-1523874fe28c
  Args:
    a: 10
    b: 5
================================= Tool Message =================================
Name: multiply

50


In [48]:
## Stategraph
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

## Node definition
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

## Grpah
builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

## Add Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "tool_calling_llm")

## compile the graph
graph = builder.compile()

In [50]:
response = graph.invoke(
    {"messages": "Tell me 1000 * 3 and later add 5 with 10"}
)
for m in response["messages"]:
    m.pretty_print()


================================ Human Message =================================

Tell me 1000 * 3 and later add 5 with 10
================================== Ai Message ==================================
Tool Calls:
  multiply (26b13e20-96d4-4a7f-8817-09cff67204ac)
 Call ID: 26b13e20-96d4-4a7f-8817-09cff67204ac
  Args:
    a: 1000
    b: 3
  addition (623d2509-9d0f-425d-abc7-1e3b38d9d913)
 Call ID: 623d2509-9d0f-425d-abc7-1e3b38d9d913
  Args:
    a: 5
    b: 10
================================= Tool Message =================================
Name: multiply

3000
================================= Tool Message =================================
Name: addition

15
================================== Ai Message ==================================

The result of the multiplication is 3000.

Adding 15 to 3000 results in: 3015.


In [51]:
## Stategraph
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()


## Node definition
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


## Grpah
builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

## Add Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "tool_calling_llm")

## compile the graph
graph = builder.compile(checkpointer=memory)


In [53]:
config = {"configurable": {"thread_id": "1"}}

response = graph.invoke({"messages": "Hi my name is Disen Chitilapilly Devassy"}, config=config)
for m in response["messages"]:
    m.pretty_print()


================================ Human Message =================================

Hi my name is Disen Chitilapilly Devassy
================================== Ai Message ==================================
Tool Calls:
  addition (ea3acb21-8426-4f4a-80f3-d4c8a83f1bbd)
 Call ID: ea3acb21-8426-4f4a-80f3-d4c8a83f1bbd
  Args:
    a: 0
    b: 13
================================= Tool Message =================================
Name: addition

13
================================== Ai Message ==================================

Hello Disen! The answer to the addition question you submitted is 13. Is there anything else I can help you with?
================================ Human Message =================================

Hi my name is Disen Chitilapilly Devassy
================================== Ai Message ==================================
Tool Calls:
  addition (3394e4f3-9b42-4029-ac8b-0f9d40b8ef1d)
 Call ID: 3394e4f3-9b42-4029-ac8b-0f9d40b8ef1d
  Args:
    a: 0
    b: 13
========================

In [54]:
response = graph.invoke({"messages": "Hey what is my name"}, config=config)
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Hi my name is Disen Chitilapilly Devassy
================================== Ai Message ==================================
Tool Calls:
  addition (ea3acb21-8426-4f4a-80f3-d4c8a83f1bbd)
 Call ID: ea3acb21-8426-4f4a-80f3-d4c8a83f1bbd
  Args:
    a: 0
    b: 13
================================= Tool Message =================================
Name: addition

13
================================== Ai Message ==================================

Hello Disen! The answer to the addition question you submitted is 13. Is there anything else I can help you with?
================================ Human Message =================================

Hi my name is Disen Chitilapilly Devassy
================================== Ai Message ==================================
Tool Calls:
  addition (3394e4f3-9b42-4029-ac8b-0f9d40b8ef1d)
 Call ID: 3394e4f3-9b42-4029-ac8b-0f9d40b8ef1d
  Args:
    a: 0
    b: 13
========================

In [56]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()


In [57]:
def superbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


In [58]:
graph = StateGraph(State)

## node
graph.add_node("SuperBot", superbot)
## Edges

graph.add_edge(START, "SuperBot")
graph.add_edge("SuperBot", END)


graph_builder = graph.compile(checkpointer=memory)


In [62]:
# Create a thread
config = {"configurable": {"thread_id": "3"}}

for chunk in graph_builder.stream(
    {"messages": "Hi,My name is Krish And I like cricket"},
    config,
    stream_mode="updates",
):
    print(chunk)


{'SuperBot': {'messages': [AIMessage(content="I think we've had this conversation before, Krish! You're telling me again that you like cricket. That's okay, I'm happy to chat with you about it!\n\nIf you don't mind me asking, what do you like most about the game of cricket? Is it the skill and strategy involved in playing it, or maybe the excitement of watching a match?\n\n(By the way, I think we might need to mix up the conversation a bit. Do you want to talk about something new, or keep chatting about cricket?)", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-08-24T12:30:07.811212Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3872069291, 'load_duration': 122017291, 'prompt_eval_count': 293, 'prompt_eval_duration': 830805000, 'eval_count': 107, 'eval_duration': 2915903750, 'model_name': 'llama3.2'}, id='run--fe86544e-b462-4912-bae7-03ced14cc1cf-0', usage_metadata={'input_tokens': 293, 'output_tokens': 107, 'total_tokens': 400})]}}


In [63]:
# Create a thread
config = {"configurable": {"thread_id": "4"}}

for chunk in graph_builder.stream(
    {"messages": "Hi,My name is Krish And I like cricket"},
    config,
    stream_mode="updates",
):
    print(chunk)


{'SuperBot': {'messages': [AIMessage(content="Namaste Krish! Nice to meet you! Cricket is a great sport, isn't it? Which team do you support, and who's your favorite player?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-08-24T12:31:06.77999Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1719362708, 'load_duration': 95588917, 'prompt_eval_count': 35, 'prompt_eval_duration': 747135000, 'eval_count': 33, 'eval_duration': 876002375, 'model_name': 'llama3.2'}, id='run--fec5ba6a-f75e-403d-99e7-701c9d67ea50-0', usage_metadata={'input_tokens': 35, 'output_tokens': 33, 'total_tokens': 68})]}}


In [64]:
for chunk in graph_builder.stream(
    {"messages": "I also like football"}, config, stream_mode="values"
):
    print(chunk)


{'messages': [HumanMessage(content='Hi,My name is Krish And I like cricket', additional_kwargs={}, response_metadata={}, id='6f14fcc0-3bd7-4d5f-87ab-34234f69fafb'), AIMessage(content="Namaste Krish! Nice to meet you! Cricket is a great sport, isn't it? Which team do you support, and who's your favorite player?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-08-24T12:31:06.77999Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1719362708, 'load_duration': 95588917, 'prompt_eval_count': 35, 'prompt_eval_duration': 747135000, 'eval_count': 33, 'eval_duration': 876002375, 'model_name': 'llama3.2'}, id='run--fec5ba6a-f75e-403d-99e7-701c9d67ea50-0', usage_metadata={'input_tokens': 35, 'output_tokens': 33, 'total_tokens': 68}), HumanMessage(content='I also like football', additional_kwargs={}, response_metadata={}, id='8c55738b-95ca-45be-93a7-2c12763fd6f6')]}
{'messages': [HumanMessage(content='Hi,My name is Krish And I like cricket', additional_kw

In [65]:
config = {"configurable": {"thread_id": "5"}}

async for event in graph_builder.astream_events(
    {"messages": ["Hi My name is Krish and I like to play cricket"]},
    config,
    version="v2",
):
    print(event)


{'event': 'on_chain_start', 'data': {'input': {'messages': ['Hi My name is Krish and I like to play cricket']}}, 'name': 'LangGraph', 'tags': [], 'run_id': 'd937d269-53c1-4b72-8e8a-d62ea827782a', 'metadata': {'thread_id': '5'}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='Hi My name is Krish and I like to play cricket', additional_kwargs={}, response_metadata={}, id='44fe9f9b-03ed-48af-8288-eaa7edb83daa')]}}, 'name': 'SuperBot', 'tags': ['graph:step:1'], 'run_id': '76dc6793-15ce-4d49-842c-c4124238997b', 'metadata': {'thread_id': '5', 'langgraph_step': 1, 'langgraph_node': 'SuperBot', 'langgraph_triggers': ('branch:to:SuperBot',), 'langgraph_path': ('__pregel_pull', 'SuperBot'), 'langgraph_checkpoint_ns': 'SuperBot:568dc40d-4eda-2dff-82fb-705c6562ab3b'}, 'parent_ids': ['d937d269-53c1-4b72-8e8a-d62ea827782a']}
{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='Hi My name is Krish and I like to pla